# Data Challenge 3, Costly Conversion
## Liz Norred

Summary:

Methods:

Results:

Recommendations:

### Import Libraries and Data

In [4]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
from timezonefinder import TimezoneFinder

import seaborn as sns

We run into problems right away when the datetime values can't be inferred. 3% of the dataset contains these values. Roughly the same number of test cases (3.3% for low and 3.2% for high) contain these NaT values, so let's just exclude them.

In [5]:
test = pd.read_csv(r'C:\Users\eliza\dev\Python\IDS-Data-Challenges/test_results.csv', index_col='user_id',header = 0, parse_dates=['timestamp'],infer_datetime_format=True)
time_user= pd.to_datetime(test.timestamp.values, format="%Y-%m-%d %H:%M:%S", errors='coerce')
test['time_user']=time_user
test=test.drop(columns='timestamp')
test=test.dropna()
print(test.shape)
ProfileReport(test)

(306529, 7)


Number of variables,8
Number of observations,306529
Total Missing (%),0.0%
Total size in memory,18.7 MiB
Average record size in memory,64.0 B
Numeric,1
Categorical,3
Boolean,2
Date,1
Text (Unique),0
Rejected,1


In [6]:
users = pd.read_csv(r'C:\Users\eliza\dev\Python\IDS-Data-Challenges/user_table.csv', index_col='user_id',header = 0)
print(users.shape)
ProfileReport(users)

(275616, 4)


Number of variables,5
Number of observations,275616
Total Missing (%),0.0%
Total size in memory,10.5 MiB
Average record size in memory,40.0 B
Numeric,3
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,1


All values in the users table are in the US, so this column can be excluded. The test column appears to be a one-hot encoded version of either 39 or 59 dollar pricing. The datetime values are in user time, not standard UTC time, so if we want to look at them in aggregate over time, this will need to be converted-- TimezoneFinder can do this for us without an online lookup, since we have the lat/long values. So let's join these tables (knowing that since the tables are not the same size, we'll have some NaN values in the geographical data.

In [7]:
test=test.join(users)
test.drop(columns=['country','price'])

,source,device,operative_system,test,converted,time_user,city,lat,long
user_id,,,,,,,,,
604839,ads_facebook,mobile,iOS,0,0,2015-05-08 03:38:34,Buffalo,42.89,-78.86
624057,seo-google,mobile,android,0,0,2015-05-10 21:08:46,Lakeville,44.68,-93.24
317970,ads-bing,mobile,android,0,0,2015-04-04 15:01:23,Parma,41.38,-81.73
685636,direct_traffic,mobile,iOS,1,0,2015-05-07 07:26:01,Fayetteville,35.07,-78.90
820854,ads_facebook,web,mac,0,0,2015-05-24 11:04:40,Fishers,39.95,-86.02
169971,ads-google,mobile,iOS,0,0,2015-04-13 12:07:08,New York,40.67,-73.94
600150,seo_facebook,web,windows,0,0,2015-03-04 14:45:44,NaN,NaN,NaN
798371,ads-bing,mobile,android,1,1,2015-03-15 08:19:29,East Orange,40.77,-74.21
447194,ads_facebook,web,windows,1,0,2015-03-28 12:28:10,Dayton,39.78,-84.20


### Looking at the two different conditions

In [8]:
lowprice=test[test['test'] == 0]
highprice=test[test['test'] == 1]

In [9]:
ProfileReport(lowprice)

Number of variables,12
Number of observations,196073
Total Missing (%),4.3%
Total size in memory,18.0 MiB
Average record size in memory,96.0 B
Numeric,3
Categorical,5
Boolean,2
Date,1
Text (Unique),0
Rejected,1


In [10]:
ProfileReport(highprice)

Number of variables,12
Number of observations,110456
Total Missing (%),4.3%
Total size in memory,10.1 MiB
Average record size in memory,96.0 B
Numeric,3
Categorical,5
Boolean,2
Date,1
Text (Unique),0
Rejected,1


For the high price, 1715(1.6%) converted and 108741 didn't convert out of 110456. For the low price, 3905(2.0%) converted and 192168 didn't convert out of 196073. This suggests that the lower price performed better overall in conversions than the higher price, but values are not yet normalized by category. A higher proportion of users in the low price used mobile devices( high, 59.4%; low, 57.8%). This % change was also visible in the operative_system category. When I ran this initially with the faulty timestamps included, the low price category acquired 1% more users from direct traffic. This disappears in the corrected data, suggesting this error is associated with web traffic. There initally do not appear to be dramatic differences in the test samples-- but there are some mistakes in the price data! Since we don't know which category is correct between price and test, we should delete these samples. Deleting them does not noticably change the conversion numbers. 

In [11]:
# Delete users that have incompatible price/test data
lowprice= lowprice[lowprice.price != 59]
highprice= highprice[highprice.price != 39]

It may be helpful to reorganize these users into state or regional groups. I can tell there are subtle differences in the lat/long data, but this may be easier to see organized into territories. I can also convert the local signup time to   UTC values.

In [30]:
tf = TimezoneFinder()

tz_low=[]
for x in range(len(lowprice.lat.values)):
    latL= lowprice.lat.values[x]
    longL= lowprice.long.values[x]
    val=tf.timezone_at(lng=longL, lat=latL)
    print(val)
    tz_low.append(val)



America/New_York
America/Chicago
America/New_York
America/Indiana/Indianapolis
America/New_York


ValueError: cannot fit 'int' into an offset-sized integer

In [29]:
tz_low

['America/New_York',
 'America/Chicago',
 'America/New_York',
 'America/Indiana/Indianapolis',
 'America/New_York']